In [ ]:
##12
import random

from fastai import *
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re,string,unicodedata

#FastAI
import fastai
from fastai import *
from fastai.text import *
import torch
#Functional Tool
from functools import partial
#NLTK
import nltk
from nltk.tokenize.toktok import ToktokTokenizer

#SK Learn libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn import metrics
from fastai.vision.all import *
from IPython.display import Image
from torch.nn import CrossEntropyLoss

In [2]:
gendata_V1 = pd.read_csv("/Users/holee/Desktop/train.csv")

In [3]:
list_all = []
for i in gendata_V1["protein_sequence"]:
  list_all.append(" ".join(i))


In [4]:
total_join = "".join(list_all)#把 iterable 中的所有字符串元素用空字符串粘

In [5]:
unique_word = list(set(total_join))
aaa = []
for i in gendata_V1["protein_sequence"]:
     aaa.append(L([*i]))

In [7]:
def translator(x):
    temp_list = []
    for i in x:
        for j in range(len(unique_word)):
            if i == unique_word[j]:
                temp_list.append(j)
    return L(temp_list)

gendata_V1["fenkai_mum"] = gendata_V1["fenkai"].apply(translator)


In [8]:
len(gendata_V1["fenkai_mum"])

31390

In [16]:
new_data = gendata_V1["fenkai_mum"].iloc[0:30]
new_data

0                            [7, 7, 7, 7, 6, 7, 7, 7, 12, 7, 12, 12, 5, 13, 7, 4, 13, 8, 8, 2, 1, 19, 12, 4, 7, 5, 19, 9, 15, 4, 10, 9, 8, 10, 9, 12, 13, 9, 6, 5, 2, 5, 8, 12, 8, 5, 17, 1, 6, 2, 7, 5, 2, 2, 4, 2, 8, 14, 3, 5, 7, 13, 1, 15, 7, 10, 8, 9, 10, 7, 0, 13, 2, 9, 12, 13, 5, 5, 13, 5, 8, 5, 8, 8, 14, 6, 4, 5, 12, 5, 8, 4, 8, 5, 13, 4, 7, 1, 16, 4, ...]
1                    [7, 7, 7, 2, 5, 13, 4, 12, 3, 16, 13, 13, 13, 9, 7, 5, 7, 5, 15, 8, 5, 9, 0, 12, 4, 15, 13, 0, 13, 13, 15, 9, 14, 5, 0, 9, 4, 9, 9, 9, 9, 2, 12, 5, 0, 9, 12, 15, 7, 15, 9, 9, 7, 15, 9, 8, 7, 19, 13, 2, 5, 2, 13, 16, 8, 5, 15, 14, 8, 1, 4, 7, 15, 13, 13, 13, 5, 1, 13, 6, 4, 7, 13, 8, 3, 4, 14, 5, 6, 1, 5, 7, 6, 6, 12, 9, 6, 12, 13, 13, ...]
2                     [7, 7, 7, 10, 0, 14, 4, 9, 7, 14, 0, 18, 9, 1, 12, 0, 0, 7, 5, 0, 5, 0, 14, 9, 7, 2, 7, 4, 15, 8, 9, 9, 12, 3, 14, 14, 9, 2, 12, 12, 7, 6, 2, 18, 18, 7, 14, 12, 5, 8, 7, 6, 16, 7, 16, 5, 6, 2, 1, 6, 6, 7, 18, 18, 15, 12, 7, 6, 6, 18, 3, 4, 2, 14, 16, 6

In [17]:
idx = np.array(list((set(range(len(new_data))))))
np.random.shuffle(idx)
idx[:int(len(new_data)*0.8)].tolist()

[29,
 6,
 18,
 17,
 9,
 7,
 4,
 2,
 19,
 11,
 23,
 14,
 16,
 22,
 8,
 0,
 20,
 26,
 3,
 24,
 5,
 25,
 15,
 27]

In [24]:
split = [train_idx.tolist(),test_idx.tolist()]

In [25]:
import random
from torch import tensor
from torch.utils.data import Dataset

In [26]:
class data_add(nn.Module):##创建class只传入init里面变量data_add(dataset=gendata_V1,split=split,is_test=False)
    def __init__(self,dataset,split,long_sentence = 10,is_test = False):
        self.is_test = is_test
        self.dataset = dataset
        self.split = split
        self.long_sentence = long_sentence

    def __len__(self):
        if not self.is_test:
            return len(self.split[0])
        else:
            return len(self.split[1])

    def __getitem__(self, idx):
        if not self.is_test:
            idx_pool = self.split[0]
        else:
            idx_pool = self.split[1]

        df_idx = idx_pool[idx]  # 这里取正确索引
        full_sentence = self.dataset["fenkai_mum"].iloc[df_idx]

        if len(full_sentence) <= self.long_sentence + 1:
            templist = (self.long_sentence + 1)*[1000]
            templist[:self.long_sentence] = full_sentence
            x = templist[0:self.long_sentence]
            y = templist[1:self.long_sentence+1]
        else:
            x = full_sentence[0:self.long_sentence]
            y = full_sentence[1:self.long_sentence+1]

        new_y = 10 * [100]  # padding value 100 as in original
        # Copy y elements into new_y (truncate or pad)
        new_y[:len(y)] = y[:10]
        return tensor(x), tensor(new_y)




data_train_1 = data_add(dataset=gendata_V1,split=split,is_test=False)
data_test_1 = data_add(dataset=gendata_V1,split=split,is_test=True)



dl_train = DataLoader(data_train_1,batch_size=10)
dl_test = DataLoader(data_test_1,batch_size=10)
dls = DataLoaders(dl_train,dl_test)

(tensor([ 1,  5,  5, 17,  8,  3,  9, 18, 14,  6]),
 tensor([ 5,  5, 17,  8,  3,  9, 18, 14,  6,  0]))

(tensor([ 1,  5,  5, 17,  8,  3,  9, 18, 14,  6]),
 tensor([ 5,  5, 17,  8,  3,  9, 18, 14,  6,  0]))

In [27]:
dls.one_batch()

(tensor([[ 7,  7,  7,  0,  5, 12,  9, 14,  7,  1],
         [ 7,  7, 12,  7, 16,  0,  2,  1,  0,  8],
         [ 7,  7,  7, 14,  6,  0,  5,  4,  9,  9],
         [ 7,  7, 12,  7, 12,  5, 12,  4,  7, 10],
         [ 7,  7, 16,  0,  7,  7,  8, 17, 13, 12],
         [ 7,  7,  6,  0,  5,  2,  7, 13, 13,  7],
         [ 7,  7, 11, 10, 19,  9,  9, 14,  8,  1],
         [ 7,  7,  6,  5,  2, 11,  5,  5,  0,  5],
         [ 7,  7,  6,  8,  4,  1,  0,  1, 18, 18],
         [ 7,  7, 17, 16,  4, 13, 18,  2, 18, 12]]),
 tensor([[ 7,  7,  0,  5, 12,  9, 14,  7,  1,  4],
         [ 7, 12,  7, 16,  0,  2,  1,  0,  8,  7],
         [ 7,  7, 14,  6,  0,  5,  4,  9,  9, 15],
         [ 7, 12,  7, 12,  5, 12,  4,  7, 10, 19],
         [ 7, 16,  0,  7,  7,  8, 17, 13, 12, 12],
         [ 7,  6,  0,  5,  2,  7, 13, 13,  7,  9],
         [ 7, 11, 10, 19,  9,  9, 14,  8,  1,  4],
         [ 7,  6,  5,  2, 11,  5,  5,  0,  5, 11],
         [ 7,  6,  8,  4,  1,  0,  1, 18, 18, 13],
         [ 7, 17, 16,  4, 13,

In [28]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [35]:
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self. pred[0]

In [36]:
learn = Learner(dls,model,loss_func=CrossEntropyLossFlat(),cbs=[DropOutput], metrics=Perplexity())

In [37]:
learn.fit(1)

epoch,train_loss,valid_loss,perplexity,time
0,4.626028,5.541588,255.082703,00:52


In [ ]:
%%sql
